## Выделение аномалией по эффективности в процессах.

In [17]:
import pandas as pd
import numpy as np

#Подготовка данных к обработке
df = pd.read_csv('Event_log.txt', sep='\t', encoding='cp1251') 
df['Event end'] = pd.to_datetime(df['Event end']) 

#Подготовка данных к обработке: сортировка значений
order = ['Создание Заявки',
         'Заявка согласована',
         'Заявка изменена: дата доставки',
         'Заявка изменена: закупочная организация',
         'Заявка изменена: количество',
         'Заявка изменена: увеличена цена',
         'Заказ на поставку создан',
         'Требование авансового платежа',
         'Требование авансового платежа-Возврат',
         'Авансовый платеж',
         'Перерасчет по требованию авансового платежа',
         'Перерасчет авансового платежа',
         'Перерасчет авансового платежа-Возврат',
         'Заказ на поставку изменен: валюта',
         'Заказ на поставку изменен: налоговые условия',
         'Заказ на поставку изменен: срока предоставления скидки',
         'Заказ на поставку изменен: условия оплаты',
         'Заказ на поставку изменен: увеличено количество',
         'Заказ на поставку изменен: уменьшено количество',
         'Заказ на поставку изменен: увеличена цена',
         'Заказ на поставку изменен: уменьшена цена',
         'Заказ на поставку изменен: увеличена стоимость',
         'Заказ на поставку изменен: уменьшена стоимость',
         'Заказ на поставку изменен: эффективная стоимость',
         'Заказ на поставку изменен: группа закупки',
         'Заказ на поставку изменен: завод',
         'Заказ на поставку изменен: запланированный срок доставки в днях',
         'Заказ на поставку изменен: лимит на недопоставку',
         'Заказ на поставку изменен: лимит на сверх-поставку',
         'Заказ на поставку изменен: материал',
         'Заказ на поставку изменен: поставщик',
         'Заказ на поставку изменен: статус наличия счета',
         'Заказ на поставку изменен: тип документа',
         'Заказ на поставку согласован 1',
         'Заказ на поставку: изменен статус выпуска: ДанныеОтпр, возможны изменения',
         'Заказ на поставку: изменен статус выпуска: НачКод, возможны изменения',
         'Заказ на поставку: изменен статус выпуска: ДанныеСогл, измен не возможны',
         'Заказ на поставку: изменен статус выпуска: Согл, измен не возможны',
         'Заказ на поставку согласован 2',
         'Заказ на поставку: согласование отклонено',
         'Заявка: согласование отклонено',
         'Заказ на поставку бессрочно заблокирован',
         'Заказ на поставку удален',
         'Заявка удалена',
         'Заявка восстановлена',
         'Заказ на поставку восстановлен',
         'Поступление материала-Получение-Завершающая поставка',
         'Поступление материала-Получение-Частичная поставка',
         'Поступление материала-Возврат-Частичная поставка',
         'Поступление материала-Возврат-Завершающая поставка',
         'Счет предварительно полностью зарегистрирован',
         'Счет заведен',
         'Поступление счета',
         'Поступление счета-Возврат',
         'Счет блокирован: несоответствие даты',
         'Счет блокирован: несоответствие количества',
         'Счет блокирован: несоответствие цены',
         'Счет изменен: налоговые условия',
         'Счет изменен: условия оплаты',
         'Счет изменен: дата',
         'Платеж (выравнивание)',
         'Ведомость учета работ/услуг']
df['Activity'] = pd.Categorical(df['Activity'], categories=order)
df = df.sort_values(by=['Event end', 'Activity'])

#Обработка данных: удаление выравнивания
new_df = df.drop(np.where(df['Activity'] == 'Платеж (выравнивание)')[0])

#Обработка данных:группировка событий в кейсы
cases = df['CaseID'].unique()
classes = df['Activity Class'].unique()

#Анализ данных: построение матрицы описания цеопочек
features = np.zeros((len(cases), len(classes)))
for i in range(len(cases)):
    chain = df[df['CaseID'] == cases[i]]    
    chain_count = chain.groupby('Activity Class')['Activity'].value_counts()
    for j in range(len(classes)):
        if classes[j] in chain_count:
            features[i][j] = chain_count[classes[j]].sum()

#Анализ данных: подсчет среднего значения повторений для каждого класса
mean_values = np.zeros(len(classes))
tran_feat = features.T
for i in range(len(classes)):
    mean_values[i] = tran_feat[i].mean()
fin_dict = dict(zip(classes, mean_values))

#Идентификация аномальный кейсов
anomaly = np.zeros((len(cases), len(classes)))
for j in range(len(classes)):
    anomaly[np.where(tran_feat[j] > mean_values[j] + 2 * tran_feat[j].std())[0]] = 1

#Вывод 
amount_anomaly_cases = len(np.unique(np.where(anomaly == 1)[0])) 
print("Количество кейсов, в которых выявлены аномалии:", amount_anomaly_cases)
print('Отношение кейсов, в которых выявлены аномалии, к общему числу всех кейсов: ' + '{:,.2f}'.format((amount_anomaly_cases / len(cases)) * 100) + '%')

Количество кейсов, в которых выявлены аномалии: 1670
Отношение кейсов, в которых выявлены аномалии, к общему числу всех кейсов: 20.81%
